<h1>Portfolio Monitor</h1>
<h2>Updated portfolio and stock market evaluation</h2>




<h6>
    <p>VERSION: 1.0</p>
    <p>Stock-Monitor Update from Swissquote Transaction file</p>
    <p>Roberto Schlatter</p>
    <p>19-07-2021</p>
</h6>

<h3>
Prerequisite
</h3>

In [61]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
import datetime
import pickle
import os.path
import os
import csv
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.protobuf import service


<h3>
Upload to Google Sheet Function
</h3>

In [62]:
def storeDataToGoogleSheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME, df_gold, sOrientation, bSingleValue):
    def Create_Service(client_secret_file, api_service_name, api_version, *scopes):
        global service
        SCOPES = [scope for scope in scopes[0]]
        #print(SCOPES)
        cred = None

        if os.path.exists('token_write.pickle'):
            with open('token_write.pickle', 'rb') as token:
                cred = pickle.load(token)

        if not cred or not cred.valid:
            if cred and cred.expired and cred.refresh_token:
                cred.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES)
                cred = flow.run_local_server()

            with open('token_write.pickle', 'wb') as token:
                pickle.dump(cred, token)

        try:
            service = build(api_service_name, api_version, credentials=cred)
            print(api_service_name, 'service created successfully')
            #return service
        except Exception as e:
            print(e)
            #return None

    # change 'my_json_file.json' by your downloaded JSON file: my_credentials.json
    Create_Service('my_credentials.json', 'sheets', 'v4',['https://www.googleapis.com/auth/spreadsheets'])

    if(bSingleValue):
        def SingleValue_Export_Data_To_Sheets():
            response_date = service.spreadsheets().values().update(
                spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                valueInputOption='RAW',
                range=SAMPLE_RANGE_NAME,
                body=dict(
                    #Orientation: ROWS oder COLUMNS
                    majorDimension=sOrientation,
                    values=df_gold.T.reset_index().T.values.tolist())
            ).execute()
            print('Sheet successfully Updated')
        SingleValue_Export_Data_To_Sheets()
    else:
        def Export_Data_To_Sheets():
            response_date = service.spreadsheets().values().update(
                spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                valueInputOption='RAW',
                range=SAMPLE_RANGE_NAME,
                body=dict(
                    #Orientation: ROWS oder COLUMNS
                    majorDimension=sOrientation,
                    values=df_gold.T.reset_index().T.values.tolist())
            ).execute()
            print('Sheet successfully Updated')
        Export_Data_To_Sheets()

  <h3>Pre Definition for Test run and final run</h3>

In [63]:
sTestRun = False

if sTestRun:
    sGoogleSheet = "17iBlVZySEFsqI9RjPR6hVrlD_QtQ-zc46bKuar0nFOk"
else:
    sGoogleSheet = "1RX2lJwZYWUO_sJUME1M9CsMFkGQNEqMCiY2Y6vCZjF0"

<h3>Swissquote Transaction import</h3>

In [80]:
sFilename = 'transactions-from-01012010-to-14072021.xlsx'

# Set new header names in order to remove Umlaute
lst_column_names = ['Auftrag', 'Datum', 'Transaktionen', 'Symbol', 'Name', 'ISIN', 'Anzahl', 'Stueckpreis', 'Kosten','Aufgelaufene Zinsen','Nettobetrag','Waehrung Nettobetrag', 'Nettobetrag in der Waehrung des Kontos','Saldo','Waehrung']

# Dataframe initialisieren!
df_transactions = pd.DataFrame(columns = lst_column_names)

df_transactions = df_transactions.append(pd.read_excel(sFilename, index_col=1, names=lst_column_names))

# Dataframe at the end of the cell without print shows the formatted table
df_transactions.head(10)

,Auftrag,Datum,Transaktionen,Symbol,Name,ISIN,Anzahl,Stueckpreis,Kosten,Aufgelaufene Zinsen,Nettobetrag,Waehrung Nettobetrag,Nettobetrag in der Waehrung des Kontos,Saldo,Waehrung
0,2021-07-09 11:03:34,NaN,Buy,ADA,ADA,EX0X00ADA000,150.0,1.32,1.98,0,-199.98,USD,-199.98,27.55,USD
0,2021-07-05 11:22:08,NaN,Dividende,VWRL,Vanguard All World ETF Dist,IE00B3RBWM25,1.0,113.92,0.00,0,113.92,USD,113.92,227.53,USD
0,2021-07-01 13:07:48,NaN,Dividende,VFEM,Vanguard EM ETF Dist,IE00B3VVMM84,1.0,134.61,0.00,0,134.61,USD,134.61,113.61,USD
0,2021-06-30 22:19:51,NaN,Fx-Gutschrift Comp.,NaN,NaN,NaN,1.0,141.34,0.00,0,141.34,USD,141.34,-21.00,USD
0,2021-06-30 22:19:51,NaN,Fx-Belastung Comp.,NaN,NaN,NaN,1.0,132.00,0.00,0,-132,CHF,-132,0.00,CHF
0,2021-06-30 20:59:16,NaN,Depotgebühren,NaN,NaN,NaN,1.0,48.00,3.70,0,-51.7,CHF,-51.7,132.00,CHF
88666265,2021-06-30 19:27:07,NaN,Kauf,NIO,NIO ADS REP 1 CL A ORD,US62914V1061,6.0,53.19,10.33,0,-329.47,USD,-329.47,-162.34,USD
0,2021-06-30 17:42:12,NaN,Dividende,VJPN,Vanguard Japan ETF Dist,IE00B95PGT31,1.0,69.43,0.00,0,69.43,USD,69.43,167.13,USD
0,2021-06-29 17:00:42,NaN,Dividende,VTI,VANGUARD TOTAL STOCK MARKET ET,US9229087690,1.0,118.18,35.46,0,82.72,USD,82.72,97.70,USD
88397267,2021-06-24 09:01:05,NaN,Kauf,VJPN,Vanguard Japan ETF Dist,IE00B95PGT31,21.0,32.89,1.05,0,-691.75,CHF,-691.75,183.70,CHF


<h5>Dividends</h5>

In [65]:
lst_div_names = ['Jahr', 'Monat', 'Symbol', 'Nettobetrag in der Waehrung des Kontos']
df_dividende = pd.DataFrame(columns = lst_div_names)
df_dividende = df_dividende.append(df_transactions[df_transactions['Transaktionen'].isin(['Dividende'])])


# Nach Datum sortiert - für Monatsübersicht
#Create new Columns with Jahr and Monat as extracted from "Datum"
df_dividende['Jahr'] = pd.DatetimeIndex(df_dividende['Auftrag']).year
df_dividende['Monat'] = pd.DatetimeIndex(df_dividende['Auftrag']).month


lst_header = ['Jahr', 'Monat', 'Symbol']

df_dividende_year_month = df_dividende.groupby(lst_header)['Nettobetrag in der Waehrung des Kontos'].sum()                                                                                                   
                                                                                                     
#lst_header_2 = ['Symbol']
#lst_header_div = ['Symbol','Jahr','Monat']

#df_export_dividende = df_dividende.groupby(lst_header_2)['Nettobetrag in der Waehrung des Kontos'].sum().head(25)

#-----------------------------------------
# Call Upload: Dividend detail table by month and year
#-----------------------------------------
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!M03', df_dividende_year_month, 'COLUMNS', False)

df_dividende_year_month

sheets service created successfully
Sheet successfully Updated


Jahr  Monat  Symbol
2012  3      ROG        22.10
      8      AAPL        9.27
      11     AAPL        9.27
2015  7      SCE        49.77
2016  1      SPY5       21.36
                        ...  
2021  6      MSFT       28.62
             VJPN       69.43
             VTI        82.72
      7      VFEM      134.61
             VWRL      113.92
Name: Nettobetrag in der Waehrung des Kontos, Length: 94, dtype: float64

In [66]:
# df_dividende['Nettobetrag in der Währung des Kontos'] = pd.to_numeric(df_dividende['Nettobetrag in der Waehrung des Kontos'])

lst_header = ['Symbol']

df_export_dividende_summary = df_dividende.groupby(lst_header)['Nettobetrag in der Waehrung des Kontos'].sum().head(25)

#-----------------------------------------
# Call Upload: Dividend summary on first page
#-----------------------------------------
storeDataToGoogleSheet(sGoogleSheet,'SUMMARY!B56', df_export_dividende_summary, 'COLUMNS', False)

df_export_dividende_summary

sheets service created successfully
Sheet successfully Updated


Symbol
AAPL       168.17
AMS         38.08
AXP         27.39
BLV         30.88
FCAU      3805.53
IBTM        24.72
MSFT       116.22
NVDA        12.82
PGHN       220.35
QCOM       101.66
ROG       1615.89
SCE         49.77
SMMCHA     316.92
SPY5        43.50
STLA       470.85
TLT        152.56
TSM        141.25
VFEM       449.45
VJPN        69.43
VTI        501.07
VWRL       401.46
WM          27.86
XDJP       140.08
Name: Nettobetrag in der Waehrung des Kontos, dtype: float64

<h5>Costs</h5>

In [67]:
df_Kosten_total = pd.DataFrame()
df_Kosten_total = df_Kosten_total.append(df_transactions[df_transactions['Transaktionen'].isin(['Kauf', 'Verkauf'])])

df_Kosten_total['Jahr'] = pd.DatetimeIndex(df_Kosten_total['Auftrag']).year
df_Kosten_total['Monat'] = pd.DatetimeIndex(df_Kosten_total['Auftrag']).month

# Final dataframe for costs
lst_header_cost = ['Transaktionen','Symbol', 'Jahr', 'Monat']
df_export_Kosten_year = df_Kosten_total.groupby(lst_header_cost)['Kosten'].sum()

#-----------------------------------------
# Call Upload: Cost per asset and month & year
#-----------------------------------------
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!A03', df_export_Kosten_year, 'COLUMNS',False)

df_export_Kosten_year

sheets service created successfully
Sheet successfully Updated


Transaktionen  Symbol  Jahr  Monat
Kauf           AAPL    2012  3        40.34
                       2016  2        42.43
                       2018  10       35.88
                             11       28.37
                       2019  1        27.59
                                      ...  
Verkauf        VTI     2020  3        14.93
               WM      2020  9         8.37
               XDJP    2020  9        72.02
               XMCH    2020  4        46.85
               ZM      2020  4         8.39
Name: Kosten, Length: 224, dtype: float64

In [68]:
lst_header_cost = ['Symbol']
df_export_Kosten_total = df_Kosten_total.groupby(lst_header_cost)['Kosten'].sum()

#-----------------------------------------
# Call Upload: Cost per asset in total
#-----------------------------------------
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!G03', df_export_Kosten_total, 'COLUMNS',False)

df_export_Kosten_total

sheets service created successfully
Sheet successfully Updated


Symbol
AAPL    464.58
ABNB     68.94
ACIC     75.97
ACIU     42.40
ACWI     71.65
         ...  
VWRL    114.85
WM       87.66
XDJP    217.97
XMCH    139.50
ZM       47.12
Name: Kosten, Length: 66, dtype: float64

<h5>Total Performance</h5>

In [69]:
df_Performance_assets = pd.DataFrame()
df_Performance_buy = pd.DataFrame()
df_Performance_sell = pd.DataFrame()

df_Performance_buy = df_Performance_buy.append(df_transactions[df_transactions['Transaktionen'].isin(['Kauf'])])

lst_header_perf = ['Symbol']

# Remove tausender Trennzeichen um Kolonne als Zahlenwert beim Summieren zu verewenden
df_Performance_buy = df_Performance_buy.groupby(lst_header_perf)['Nettobetrag in der Waehrung des Kontos'].sum()

df_Performance_sell = df_Performance_sell.append(df_transactions[df_transactions['Transaktionen'].isin(['Verkauf'])])

# Remove tausender Trennzeichen um Kolonne als Zahlenwert beim Summieren zu verewenden
df_Performance_sell['Nettobetrag in der Währung des Kontos'] = df_Performance_sell['Nettobetrag in der Waehrung des Kontos'].replace("'")

df_Performance_sell = df_Performance_sell.groupby(lst_header_perf)['Nettobetrag in der Waehrung des Kontos'].sum()

df_Performance_assets =  df_Performance_sell - (-df_Performance_buy)

df_Performance_assets.dropna(inplace=True)
#-----------------------------------------
# Call Upload: Performance per asset
#-----------------------------------------
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!R03', df_Performance_assets, 'COLUMNS', False)

##********
## Noch zum korrigieren - nur Assets welche auch komplett verkauft wurden berücksichtigen - ansonsten sind aktuelle Positionen im Minus
##********

df_Performance_assets

sheets service created successfully
Sheet successfully Updated


Symbol
AAPL       4179.35
ABNB       -254.44
ACIC        -80.97
ACIU       -130.56
ACWI        -82.45
AMS       -1980.25
AMZN       1010.97
ASML       -527.07
ATLN       2374.70
AUUSI       298.25
AXP        1043.32
BABA      -4551.54
BB          -31.75
BLUE        -76.14
BLV         -64.05
DOCU        221.41
FCX        -719.30
GOOG       -629.69
GUI        -133.46
IBTM        -72.39
INMD        958.24
KNDI        -67.17
LHN        -133.10
MSFT       4998.69
MU         -520.04
NESN       -162.15
NET        -156.24
NVDA      -1566.65
OBSV       -260.67
OCGN      -1723.39
OERL       -425.20
PAL         -62.35
PGHN      -6451.10
PLTR      -5315.71
QCOM       1726.78
ROG       40797.80
SCE        -481.28
SHOP      -1654.36
SIKA      -2429.10
SMMCHA   -30930.95
SPY         -50.41
SPY5         99.07
SRAIL       -21.80
SREN        -12.30
TLT        1233.50
TSLA       1740.54
TSM        2356.17
VACN        -71.05
VFEM     -18954.35
VTI      -29274.14
WM          359.46
XDJP         92.43
XMCH 

<h5>Buy, Sell and Withdrawl Balance - including Crypto</h5>

In [76]:
# get current date and format it
pUpdateDate = datetime.datetime.now()
pDay = pUpdateDate.strftime("%A")
pDate = pUpdateDate.strftime("%d-%m-%Y | %H:%M:%S")
lst_date = [pDay, pDate]

# number of current assets - KAUF / VERKAUF
df_numberAssets_k = pd.DataFrame()
df_numberAssets_count_k = pd.DataFrame()

df_numberAssets_k = df_numberAssets_k.append(df_transactions[df_transactions['Transaktionen'].isin(['Kauf'])])
#df_numberAssets_k = df_numberAssets_k[['Symbol','Anzahl']]
df_numberAssets_count_k = df_numberAssets_k.groupby('Symbol')['Anzahl'].sum()

# Portfolio: Anzahl Positionen KAUF
df_numberAssets_count_k


Symbol
AAPL    150.0
ABNB     14.0
ACIC    500.0
ACIU     38.0
ACWI      8.0
        ...  
VWRL    196.0
WM       44.0
XDJP    547.0
XMCH    500.0
ZM       35.0
Name: Anzahl, Length: 63, dtype: float64

In [125]:
df_numberAssets_v = pd.DataFrame()
df_numberAssets_count_v = pd.DataFrame()
df_current_Portfolio = pd.DataFrame()
df_quantity_adjustments = pd.DataFrame({'Symbols': ['FCAU'],'Anzahl': ['2000']})

df_numberAssets_v = df_transactions[df_transactions['Transaktionen'].isin(['Verkauf'])]
df_numberAssets_v = df_numberAssets_v[['Symbol','Anzahl']]
df_numberAssets_count_v = df_numberAssets_v.groupby('Symbol')['Anzahl'].sum()

# Aktuelles Portfolio - Fill value = 0
df_current_Portfolio = df_numberAssets_count_k.sub(df_numberAssets_count_v,fill_value=0)

df_current_Portfolio[df_current_Portfolio >0]

#-----------------------------------------
# Call Upload: Current Portfolio
#-----------------------------------------
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!BU03', df_current_Portfolio, 'COLUMNS', False)


sheets service created successfully
Sheet successfully Updated


<h3>CRYPTHO</H3>

In [ ]:
# number of current assets - CRYPTHO BUY
df_numberAssets_c = df_transactions[df_transactions['Transaktionen'].isin(['Buy'])]
df_numberAssets_c = df_numberAssets_c[['Symbol','Anzahl']]
df_numberAssets_count_c = df_numberAssets_c.groupby('Symbol')['Anzahl'].sum()
print(df_numberAssets_count_c)
# Summe 'Nettobetrag' noch ausrechnen
df_numberAssets_c_nettobetrag = df_transactions[df_transactions['Transaktionen'].isin(['Buy'])]
df_numberAssets_c_nettobetrag = df_numberAssets_c_nettobetrag[['Symbol','Anzahl','Nettobetrag']]

#  Convert 'Nettobetrag' Column to abs values
df_numberAssets_count_c_nettobetrag = df_numberAssets_c_nettobetrag.groupby('Symbol').Nettobetrag.apply(lambda c: c.abs().sum())

print("Nettobetrag Cryptho sum():", df_numberAssets_count_c_nettobetrag)

# number of current assets - CRYPTHO WITHDRAWAL
df_numberAssets_w = df_transactions[df_transactions['Transaktionen'].isin(['Withdrawal'])]
df_numberAssets_w = df_numberAssets_w[['Symbol','Anzahl']]
df_numberAssets_count_w = df_numberAssets_w.groupby('Symbol')['Anzahl'].sum()

df_numberAssets_count_w = pd.DataFrame(df_numberAssets_count_w)
df_numberAssets_count_w.fillna("0")

df_numberAssets_sell = df_transactions[df_transactions['Transaktionen'].isin(['Sell'])]
df_numberAssets_sell = df_numberAssets_sell[['Symbol','Anzahl']]
df_numberAssets_count_sell = df_numberAssets_sell.groupby('Symbol')['Anzahl'].sum()

df_numberAssets_count_sell = pd.DataFrame(df_numberAssets_count_sell)
df_numberAssets_count_sell.fillna("0")

df_numberAssets_count_w = df_numberAssets_count_w + df_numberAssets_count_sell

print("Aktueller Cryptho Bestand_w: ", df_numberAssets_count_w)

print("Aktueller Cryptho Bestand_k ", df_numberAssets_count_sell )

# ZU beachten: Bei Aktiensplitt stimmt die finale Anzahl nicht
# Wenn ein Asset noch immer gehalten wird - sprich keine Verkäufe getätigt wurden - steht im finalen Dataframe NaN
# --> dies muss abgefangen werden

df_numberAssets_count_actual = df_numberAssets_count_k.sub(df_numberAssets_count_v)

print(df_numberAssets_count_actual)

<h5>Investment Balance</h5>

In [132]:
# EINZAHLUNGEN - 'VERGÜTUNG' pro Jahr
df_sumInvestment = df_transactions[df_transactions['Transaktionen'].isin(['Vergütung'])]
df_sumInvestment['Jahr'] = pd.DatetimeIndex(df_sumInvestment.iloc[:,0]).year
df_sumInvestment = df_sumInvestment[['Jahr', 'Nettobetrag']]
df_sumInvestment_total = df_sumInvestment.groupby('Jahr')['Nettobetrag'].sum()

print(df_sumInvestment_total)

# AUSZAHLUNGEN - 'Auszahlung' pro Jahr
df_sumInvestment_out = df_transactions[df_transactions['Transaktionen'].isin(['Auszahlung'])]
df_sumInvestment_out['Jahr'] = pd.DatetimeIndex(df_sumInvestment_out.iloc[:,0]).year
df_sumInvestment_out = df_sumInvestment_out[['Jahr', 'Nettobetrag']]
df_sumInvestment_out_total = df_sumInvestment_out.groupby('Jahr')['Nettobetrag'].sum()

df_sumInvestment_total_net=df_sumInvestment_total.sub(df_sumInvestment_out_total,fill_value=0)

#-----------------------------------------
# Call Upload: Investments per Year
#-----------------------------------------
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!BO03', df_sumInvestment_total_net, 'COLUMNS', False)



<ipython-input-132-f62366f96128>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sumInvestment['Jahr'] = pd.DatetimeIndex(df_sumInvestment.iloc[:,0]).year
<ipython-input-132-f62366f96128>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sumInvestment_out['Jahr'] = pd.DatetimeIndex(df_sumInvestment_out.iloc[:,0]).year


Jahr
2011     1160.00
2012     4500.00
2014    18750.00
2015     6500.00
2016    10950.00
2017     7500.00
2018     8000.00
2019    33398.27
2020    60000.00
2021    20000.00
Name: Nettobetrag, dtype: float64
sheets service created successfully
Sheet successfully Updated


<h5>Google Sheets Update Function</h5>

In [12]:
def storeDataToGoogleSheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME, df_gold, sOrientation, bSingleValue):
    def Create_Service(client_secret_file, api_service_name, api_version, *scopes):
        global service
        SCOPES = [scope for scope in scopes[0]]
        #print(SCOPES)
        cred = None

        if os.path.exists('token_write.pickle'):
            with open('token_write.pickle', 'rb') as token:
                cred = pickle.load(token)

        if not cred or not cred.valid:
            if cred and cred.expired and cred.refresh_token:
                cred.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES)
                cred = flow.run_local_server()

            with open('token_write.pickle', 'wb') as token:
                pickle.dump(cred, token)

        try:
            service = build(api_service_name, api_version, credentials=cred)
            print(api_service_name, 'service created successfully')
            #return service
        except Exception as e:
            print(e)
            #return None

    # change 'my_json_file.json' by your downloaded JSON file: my_credentials.json
    Create_Service('my_credentials.json', 'sheets', 'v4',['https://www.googleapis.com/auth/spreadsheets'])

    if(bSingleValue):
        def SingleValue_Export_Data_To_Sheets():
            response_date = service.spreadsheets().values().update(
                spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                valueInputOption='RAW',
                range=SAMPLE_RANGE_NAME,
                body=dict(
                    #Orientation: ROWS oder COLUMNS
                    majorDimension=sOrientation,
                    values=df_gold.T.reset_index().T.values.tolist())
            ).execute()
            print('Sheet successfully Updated')
        SingleValue_Export_Data_To_Sheets()
    else:
        def Export_Data_To_Sheets():
            response_date = service.spreadsheets().values().update(
                spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                valueInputOption='RAW',
                range=SAMPLE_RANGE_NAME,
                body=dict(
                    #Orientation: ROWS oder COLUMNS
                    majorDimension=sOrientation,
                    values=df_gold.T.reset_index().T.values.tolist())
            ).execute()
            print('Sheet successfully Updated')
        Export_Data_To_Sheets()

<h5>Google Function Call</h5>

In [13]:


# Date & time stamp of update
#storeDataToGoogleSheet(sGoogleSheet, 'SUMMARY!C02', lst_date, 'ROWS', True)





# Performance calculation buy & sell price
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!R03', df_export_performance_values, 'COLUMNS', False)


# Anzahl Assets Kauf
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!X03', df_numberAssets_count_k, 'COLUMNS', False)

# Anzahl Assets Verkauf
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!AA03', df_numberAssets_count_v, 'COLUMNS', False)

# Anzahl Assets Kauf CRYPTHO
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!AD03', df_numberAssets_count_c, 'COLUMNS', False)

# Anzahl Assets Verkauf CRYPTHO
#storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!AG03', df_numberAssets_count_w, 'COLUMNS', False)

# Anzahl Initial Value Kauf CRYPTHO
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!BG03', df_numberAssets_count_c_nettobetrag, 'COLUMNS', False)

# Anzahl Initial Value Kauf ASSETS
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!BC03', df_numberAssets_count_nettobetrag, 'COLUMNS', False)



# Auszahlungen pro Jahr
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!BR03', df_sumInvestment_out_total, 'COLUMNS', False)



sheets service created successfully


TypeError: unsupported operand type(s) for &: 'str' and 'str'

<h3>AlleAktien Freiheitsfonds Import</h3>

<h3>Chart Creation</h3>